In [3]:
import sapphire, json
import scipy as sp
import matplotlib.pylab as pl

```
Import libraries for analysis.
Aim: Import CosmicPi events in a format compatible with sapphire analysis framework
CosmicPi data stored in a json format with following tags
        Detector......: %s
        Sequence......: %s
        Barometer.....: %s
        Thermometer...: %s
        Location......: %s
        Vibration.....: %s
        Accelerometer.: %s
        Magnetometer..: %s
        Timing........: %s
        Status........: %s
        Event.........: %s
        """ % (self.detector_id, self.sequence, self.barometer, self.temperature, self.location,
               self.vibration, self.accelerometer, self.magnetometer, self.timing, self.status,
self.event if 'event' in self.__dict__ else None)
```

```
HiSPARC event objects follow the following tags.
Stations are defined as a separate object storing location, accelerometer/magnetometer data

description := {
  "event_id": UInt32Col(shape=(), dflt=0, pos=0),
  "timestamp": Time32Col(shape=(), dflt=0, pos=1),
  "nanoseconds": UInt32Col(shape=(), dflt=0, pos=2),
  "ext_timestamp": UInt64Col(shape=(), dflt=0, pos=3),
  "pulseheights": Int16Col(shape=(4,), dflt=0, pos=4),
  "integrals": Int32Col(shape=(4,), dflt=0, pos=5),
  "n1": Float32Col(shape=(), dflt=0.0, pos=6),
  "n2": Float32Col(shape=(), dflt=0.0, pos=7),
  "n3": Float32Col(shape=(), dflt=0.0, pos=8),
  "n4": Float32Col(shape=(), dflt=0.0, pos=9),
  "t1": Float32Col(shape=(), dflt=0.0, pos=10),
  "t2": Float32Col(shape=(), dflt=0.0, pos=11),
  "t3": Float32Col(shape=(), dflt=0.0, pos=12),
  "t4": Float32Col(shape=(), dflt=0.0, pos=13),
  "t_trigger": Float32Col(shape=(), dflt=0.0, pos=14)}
```


In [5]:
def cosmicPiToHiSPARC(srcFile):
    try:
        srcDat = open(srcFile, "r")
        srcInput = srcDat.readlines()
        srcDat.close()
        readData=json.read(srcInput)
        
    except:
        print "Error invalid data source"
        
def load_data_CosmicPi(file, group, dataFile, type='events'):
    """Load downloaded event summary data CosmicPi data file into HiSPARC PyTables file.

    Exact formatting of CosmicPi data file to be determined, but sample format to be modified depending
    on file structure.
    If you've previously downloaded event summary data from
    https://www.i2u2.org/elab/cosmic/data/search.jsp in TSV format, you can load them into a PyTables
    file using this method. The result is equal to directly downloading data
    using :func:`download_data`.

    TBD - How to add a destination group. CosmicPi event includes equivalent data for event group and events.
    Group can be defined on the flat, potentially new group for each event (not sure scales well with performance)

    :param file: the PyTables datafile handler
    :param group: the PyTables destination group, which need not exist
    :param tsv_file: CosmicPi event file
    :param type: the datatype to load, either 'events' or 'weather'. For now only 'events' is expected in CosmicPi 
     data.

    Example::

        >>> import tables
        >>> import sapphire.esd
        >>> data = tables.open_file('data.h5', 'w')
        >>> sapphire.esd.load_data_CosmicPi(data, '/s501', 'events-s501-20130910.tsv')

    """
    
    if type == 'events':
        table = _get_or_create_events_table_CosmicPi(file, group)
        read_and_store_class = _read_line_and_store_event_CosmicPi_class
    elif type == 'weather':    #Note, not currently functional for CosmicPi event data
        table = _get_or_create_weather_table(file, group)
        read_and_store_class = _read_line_and_store_weather_class
    else:
        raise ValueError("Data type not recognized.")

    with open(dataFile, 'rb') as data:
        reader = csv.reader(data, delimiter='\t')
        with read_and_store_class(table) as writer:
            for line in reader:
                writer.store_line(line)
                
def _get_or_create_events_table_CosmicPi(file, group):
    """Get or create event table in PyTables file"""

    try:
        return file.get_node(group, 'events')
    except tables.NoSuchNodeError:
        return _create_events_table_CosmicPi(file, group)

def _create_events_table_CosmicPi(file, group):
    """Create event table in PyTables file

    Create an event table containing the ESD data columns which are
    available in the JSON tags. CosmicPi output data is differently formatted
    to HiSPARC data therefore some data is missing/not appropriate. Default values
    are assigned for this.

    :param file: PyTables file
    :param group: the group to contain the events table, which need not
                  exist

    """
    description = {'event_id': tables.UInt32Col(pos=0),
                   'timestamp': tables.Time32Col(pos=1),
                   'nanoseconds': tables.UInt32Col(pos=2),
                   'ext_timestamp': tables.UInt64Col(pos=3),
                   'pulseheights': tables.Int16Col(pos=4, shape=4),
                   'integrals': tables.Int32Col(pos=5, shape=4),
                   'n1': tables.Float32Col(pos=6),
                   'n2': tables.Float32Col(pos=7),
                   'n3': tables.Float32Col(pos=8),
                   'n4': tables.Float32Col(pos=9),
                   't1': tables.Float32Col(pos=10),
                   't2': tables.Float32Col(pos=11),
                   't3': tables.Float32Col(pos=12),
                   't4': tables.Float32Col(pos=13),
                   't_trigger': tables.Float32Col(pos=14)}

    return file.create_table(group, 'events', description, createparents=True)

class _read_line_and_store_event_CosmicPi_class(object):

    def __init__(self, table):
        self.table = table
        self.event_counter = len(self.table)

    def __enter__(self):
        return self

    def store_line(self, line):
        # ignore comment lines
        if line[0][0] == '#':
            return 0.
        row = self.table.row

        (self.detector_id, self.sequence, self.barometer, self.temperature, self.location,
               self.vibration, self.accelerometer, self.magnetometer, self.timing, self.status,
self.event 
        
        # convert string values to correct data types or calculate values
        row['event_id'] = self.event
        row['timestamp'] = int(datetime_to_gps(datetime.strptime(str(dateStamp)+str(timeStamp), "%Y$m%d%H%M%s")))
        row['nanoseconds'] = int(nanoseconds*(1/(42.66*10.0**6)))
        row['ext_timestamp'] = int(datetime_to_gps(datetime.strptime(str(dateStamp)+str(timeStamp), "%Y$m%d%H%M%s"))) * int(1e9) + int(nanoseconds)
        row['pulseheights'] = [int(1), int(1), int(1), int(1)]
        row['integrals'] = [int(1), int(1), int(1), int(1)]
        row['n1'] = float(1)
        row['n2'] = float(1)
        row['n3'] = float(1)
        row['n4'] = float(1)
        row['t1'] = float(t1)
        row['t2'] = float(t2)
        row['t3'] = float(t3)
        row['t4'] = float(t4)
        row['t_trigger'] = float(nanoseconds)

        # store event
        row.append()

        self.event_counter += 1
        # force flush every 1e6 rows to free buffers
        if not self.event_counter % 1000000:
            self.table.flush()

        return int(timestamp)

    def __exit__(self, type, value, traceback):
        self.table.flush()
